In [1]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 3.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score , accuracy_score, recall_score , precision_score
import torch
import re
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer ,TrainingArguments


In [3]:
df_train=pd.read_csv('/content/twitter_training.csv')
df_train.columns=['ID','Entity','Label','Tweet']
df_train.head()


,ID,Entity,Label,Tweet
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [4]:
df_val=pd.read_csv('/content/twitter_validation.csv')
df_val.columns=['ID','Entity','Label','Tweet']
df_val.head()


,ID,Entity,Label,Tweet
0,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
1,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
2,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
3,4433,Google,Neutral,Now the President is slapping Americans in the...
4,6273,FIFA,Negative,Hi @EAHelp I’ve had Madeleine McCann in my cel...


In [5]:
df_train['Tweet'].dropna(inplace=True)

In [6]:
df_train=df_train.reindex(df_train.index.drop_duplicates())

In [7]:
df_train.head(20)

,ID,Entity,Label,Tweet
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...
5,2402,Borderlands,Positive,So I spent a few hours making something for fu...
6,2402,Borderlands,Positive,So I spent a couple of hours doing something f...
7,2402,Borderlands,Positive,So I spent a few hours doing something for fun...
8,2402,Borderlands,Positive,So I spent a few hours making something for fu...
9,2402,Borderlands,Positive,2010 So I spent a few hours making something f...


In [8]:
df_train['Label'].unique()

array(['Positive', 'Neutral', 'Negative', 'Irrelevant'], dtype=object)

In [9]:
df_train['Label']=df_train['Label'].replace({'Positive':1,'Negative':2,'Neutral':0,'Irrelevant':3})

In [10]:
df_val['Label']=df_val['Label'].replace({'Positive':1,'Negative':2,'Neutral':0,'Irrelevant':3})


In [11]:
def remove_pattern(input_txt, pattern):
  if isinstance(input_txt,str):
    r=re.findall(pattern,input_txt)
    for word in r:
      input_txt=re.sub(word,'',input_txt)
    return input_txt
  else:
    return input_txt

In [12]:
df_train['Clean_Tweet']=np.vectorize(remove_pattern)(df_train['Tweet'],'@[\w]*')
df_val['Clean_Tweet']=np.vectorize(remove_pattern)(df_val['Tweet'],'@[\w]*')

In [13]:
df_train['Clean_Tweet']=df_train['Clean_Tweet'].str.replace('[^a-zA-Z#]'," ")
df_val['Clean_Tweet']=df_val['Clean_Tweet'].str.replace('[^a-zA-Z#]'," ")

In [14]:
df_train['Clean_Tweet']=df_train['Clean_Tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
df_val['Clean_Tweet']=df_val['Clean_Tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

In [15]:
x=list(df_train['Clean_Tweet'])
y=list(df_train['Label'])

In [16]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.1,random_state=42)

In [17]:
df_val=df_val.drop(['ID','Entity','Tweet'],axis=1)

In [18]:
x_val=list(df_val['Clean_Tweet'])
y_val=list(df_val['Label'])

In [19]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [20]:
Model=f'cardiffnlp/twitter-roberta-base-sentiment-latest'
tokenizer=AutoTokenizer.from_pretrained(Model)
model=AutoModelForSequenceClassification.from_pretrained(Model,num_labels=4, ignore_mismatched_sizes=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpo

In [21]:
model=model.to(device)

In [22]:
x_train_tokenized=tokenizer(x_train,padding=True,truncation=True,max_length=512)
x_val_tokenized=tokenizer(x_val,padding=True,truncation=True,max_length=512)

In [23]:
class Dataset(torch.utils.data.Dataset):
  def __init__(self,encoding,labels=None):
    self.encoding = encoding
    self.labels=labels

  def __getitem__(self,idx):
    item={key: torch.tensor(val[idx]) for key,val in self.encoding.items()}
    if self.labels is not None:
      item['labels']=torch.tensor(int(self.labels[idx]))
    return item

  def __len__(self):
    return len(self.encoding['input_ids'])

In [24]:
train_dataset=Dataset(x_train_tokenized,y_train)
val_dataset=Dataset(x_val_tokenized,y_val)

In [25]:
def compute_metrics(p):
  print(type(p))
  pred,labels=p

  accuracy=accuracy_score(y_true=labels,y_pred=np.argmax(pred,axis=1))
  f1=f1_score(y_true=labels,y_pred=np.argmax(pred,axis=1),average='weighted')
  recall=recall_score(y_true=labels,y_pred=np.argmax(pred,axis=1),average='weighted')
  precision=precision_score(y_true=labels,y_pred=np.argmax(pred,axis=1),average='weighted')

  return {'accuracy':accuracy,'f1':f1,'recall':recall,'precision':precision}

In [26]:
args = TrainingArguments(
    output_dir='output',
    num_train_epochs=3,  # Reduced number of epochs
    per_device_train_batch_size=8,  # Smaller batch size if GPU memory is an issue
    gradient_accumulation_steps=2,  # Adjust if needed
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,  # Only keep the most recent checkpoint
    logging_dir='logs',
    logging_steps=50,  # Log less frequently
    fp16=True,  # Enable mixed precision training if supported
)
trainer=Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [27]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Recall,Precision
1,0.603300,0.553919,0.818819,0.816689,0.818819,0.832597
2,0.317200,0.404663,0.893894,0.893506,0.893894,0.896644
3,0.161200,0.518610,0.904905,0.904923,0.904905,0.905805


<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>


TrainOutput(global_step=10287, training_loss=0.47566239511907604, metrics={'train_runtime': 3737.837, 'train_samples_per_second': 44.029, 'train_steps_per_second': 2.752, 'total_flos': 2.621801773826352e+16, 'train_loss': 0.47566239511907604, 'epoch': 3.0})

In [28]:
trainer.evaluate()

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.5186095237731934,
 'eval_accuracy': 0.9049049049049049,
 'eval_f1': 0.9049226163358394,
 'eval_recall': 0.9049049049049049,
 'eval_precision': 0.9058048288728796,
 'eval_runtime': 2.6976,
 'eval_samples_per_second': 370.332,
 'eval_steps_per_second': 46.338,
 'epoch': 3.0}

In [29]:
d=df_train[df_train['Label']==3]
out=[]
for i in range(20):
  text=d['Tweet'].iloc[i]
  inputs=tokenizer(text,padding=True,truncation=True,return_tensors='pt').to('cuda')

  outputs=model(**inputs)
  predictions=torch.nn.functional.softmax(outputs.logits,dim=-1)
  predictions=predictions.cpu().detach().numpy()
  out.append(predictions)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [30]:
text='How the hell are we already into Halloween month?!'
inputs=tokenizer(text,padding=True,truncation=True,return_tensors='pt').to('cuda')
outputs=model(**inputs)

In [31]:
predictions=torch.nn.functional.softmax(outputs.logits,dim=-1)
predictions=predictions.cpu().detach().numpy()
print(predictions)

[[4.2871438e-04 5.0811755e-04 3.0618068e-02 9.6844518e-01]]


In [32]:
df_test = pd.DataFrame({'Clean_Tweet': x_test, 'Label': y_test})

In [33]:
df_test.head()

,Clean_Tweet,Label
0,"Hidden figures amazing internet Verizon, they ...",2
1,"anyways fortnite here,, annoyed.",2
2,"Nostalgia-wise: series, black 2/battlefield Ju...",0
3,Comment that Battlefield 5<unk> Strike0xD fail...,3
4,Gallery: American Design Basic Unforgettable p...,0


In [36]:
model_2=AutoModelForSequenceClassification.from_pretrained(Model,num_labels=4, ignore_mismatched_sizes=True)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpo

In [37]:
model_2=model_2.to(device)

In [38]:
d2=df_test[df_test['Label']==3]
out=[]
for i in range(101):
  text=d2['Clean_Tweet'].iloc[i]
  inputs=tokenizer(text,padding=True,truncation=True,return_tensors='pt').to('cuda')

  outputs=model_2(**inputs)
  predictions=torch.nn.functional.softmax(outputs.logits,dim=-1)
  predictions=predictions.cpu().detach().numpy()
  out.append(predictions)

In [39]:
pred_classes=[np.argmax(prediction) for prediction in out]
correct_pred=sum(pred==3 for pred in pred_classes)
total_pred=len(pred_classes)
accuracy1= correct_pred / total_pred
print(f'Accuracy:{accuracy1:.2f}')

Accuracy:0.22


In [40]:
d3=df_test[df_test['Label']==3]
out2=[]
for i in range(101):
  text=d3['Clean_Tweet'].iloc[i]
  inputs=tokenizer(text,padding=True,truncation=True,return_tensors='pt').to('cuda')

  outputs=model(**inputs)
  predictions=torch.nn.functional.softmax(outputs.logits,dim=-1)
  predictions=predictions.cpu().detach().numpy()
  out2.append(predictions)

In [41]:
pred_classes=[np.argmax(prediction) for prediction in out2]
correct_pred=sum(pred==3 for pred in pred_classes)
total_pred=len(pred_classes)
accuracy1= correct_pred / total_pred
print(f'Accuracy:{accuracy1:.2f}')

Accuracy:0.93
